In [113]:
from pathlib import Path
import torch
from ray.rllib.models.torch.torch_distributions import TorchDiagGaussian
from ray.tune.result import TRAINING_ITERATION
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.core import (
    COMPONENT_LEARNER,
    COMPONENT_LEARNER_GROUP,
    COMPONENT_RL_MODULE,
)
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.examples.envs.classes.multi_agent import MultiAgentPendulum
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
    NUM_ENV_STEPS_SAMPLED_LIFETIME,
)
from ray.rllib.utils.numpy import convert_to_numpy
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    check,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env
import os

import gymnasium as gym
import numpy as np
import tree  # pip install dm_tree

from ray.rllib.core import DEFAULT_MODULE_ID
from ray.rllib.core.columns import Columns
from ray.rllib.core.rl_module.rl_module import RLModule
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.numpy import convert_to_numpy, softmax
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
)

torch, _ = try_import_torch()

In [ ]:
num-predators=2
num-prey=2

In [ ]:
# 设置参数
import sys
sys.argv = [
    'notebook_script.py',
    '--enable-new-api-stack',
    '--num-agents=4',
    # 新增参数用于指定predator和prey的数量
    '--n-predators=2',
    '--n-preys=2', 
    '--checkpoint-at-end',
    '--stop-reward=200.0',
    '--checkpoint-freq=1',
]

In [87]:
parser = add_rllib_example_script_args(
    default_iters=2,
    default_timesteps=10000,
    default_reward=0.0,
)

In [88]:
# 添加新的参数解析
parser.add_argument(
    "--n-predators",
    type=int,
    default=2,
    help="Number of predator agents"
)
parser.add_argument(
    "--n-preys", 
    type=int,
    default=2,
    help="Number of prey agents"
)
parser.add_argument(
    "--use-onnx-for-inference",
    action="store_true",
    help="Whether to convert the loaded module to ONNX format and then perform "
    "inference through this ONNX model.",
)
parser.add_argument(
    "--explore-during-inference",
    action="store_true",
    help="Whether the trained policy should use exploration during action "
    "inference.",
)
parser.add_argument(
    "--num-episodes-during-inference",
    type=int,
    default=10,
    help="Number of episodes to do inference over (after restoring from a checkpoint).",
)

_StoreAction(option_strings=['--num-episodes-during-inference'], dest='num_episodes_during_inference', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, required=False, help='Number of episodes to do inference over (after restoring from a checkpoint).', metavar=None)

In [94]:

args = parser.parse_args()

# 验证参数
assert args.n_predators > 0, "Must set --n-predators > 0 when running this script!"
assert args.n_preys > 0, "Must set --n-preys > 0 when running this script!"
assert (
    args.enable_new_api_stack
), "Must set --enable-new-api-stack when running this script!"

# 计算总智能体数量
total_agents = args.n_predators + args.n_preys
print(f"参数解析完成: n_predators={args.n_predators}, n_preys={args.n_preys}, total_agents={total_agents}, algo={args.algo}")


参数解析完成: n_predators=2, n_preys=2, total_agents=4, algo=PPO


In [95]:

# 修改环境注册，传递predator和prey的数量
register_env("env", lambda _: PettingZooEnv(
    waterworld_v4.env(
        n_predators=args.n_predators,
        n_preys=args.n_preys  # 注意：这里应该是n_preys而不是n_prey
    )
))


In [97]:

# 创建新的policies字典，匹配环境中的agent命名
predator_policies = [f"predator_{i}" for i in range(args.n_predators)]
prey_policies = [f"prey_{i}" for i in range(args.n_preys)]
all_policies = predator_policies + prey_policies
print(all_policies)

['predator_0', 'predator_1', 'prey_0', 'prey_1']


In [106]:
# 创建RL module specs字典
rl_module_specs = {p: RLModuleSpec() for p in all_policies}
# print(f"创建的policies: {list(policies.keys())}")
# print(f"创建的RL module specs: {list(rl_module_specs.keys())}")

In [99]:
base_config = (
    get_trainable_cls(args.algo)
    .get_default_config()
    .environment("env")
    .multi_agent(
        # 在新API中，只需要指定policy_mapping_fn
        policies=set(all_policies),
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .training(
        vf_loss_coeff=0.005,
    )
    .rl_module(
        rl_module_spec=MultiRLModuleSpec(
            rl_module_specs=rl_module_specs,
        ),
        model_config=DefaultModelConfig(vf_share_layers=True),
    )
)

# 训练
print("开始训练...")
results = run_rllib_example_script_experiment(base_config, args, keep_ray_up=True)
print("训练完成")

2025-06-26 07:26:13,297	INFO worker.py:1747 -- Calling ray.init() again after it has already been called.


开始训练...
== Status ==
Current time: 2025-06-26 07:26:13 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_07-26-13/PPO_2025-06-26_07-26-13/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-06-26 07:26:18 (running for 00:00:05.14)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_07-26-13/PPO_2025-06-26_07-26-13/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




== Status ==
Current time: 2025-06-26 07:26:23 (running for 00:00:10.20)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_07-26-13/PPO_2025-06-26_07-26-13/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




2025-06-26 07:26:27,986	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/qrbao/ray_results/PPO_2025-06-26_07-26-13' in 0.0141s.


== Status ==
Current time: 2025-06-26 07:26:27 (running for 00:00:14.68)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_07-26-13/PPO_2025-06-26_07-26-13/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+---------------------+------------+---------------------+--------+------------------+------+-------------------+-----------------+---------------------+-----------------+---------------------+
| Trial name          | status     | loc                 |   iter |   total time (s) |   ts |   combined return |   return prey_1 |   return predator_1 |   return prey_0 |   return predator_0 |
|---------------------+------------+---------------------+--------+------------------+------+-------------------+-----------------+---------------------+-----------------+---------------------|
| PPO_env_66eb0_00000 | TERMINATED | 192.168.0.25:830892 |   

2025-06-26 07:26:28,300	INFO tune.py:1041 -- Total run time: 15.00 seconds (14.67 seconds for the tuning loop).


训练完成


In [66]:
# # 设置参数
# import sys
# sys.argv = [
#     'notebook_script.py',
#     '--enable-new-api-stack',
#     '--num-agents=4',
#     # '--env-config={"n_predators":2,"n_preys":2,"n_evaders":5,"n_obstacles":1,"n_poisons":1}',
#     '--checkpoint-at-end',
#     '--stop-reward=200.0',
#     '--checkpoint-freq=1',
# ]

In [54]:

# env = waterworld_v4.env(render_mode="human",n_predators=2,n_preys=2,n_evaders=5,n_obstacles=1,n_poisons=1)
# env.reset(seed=42)

# for agent in env.agent_iter():
#     observation, reward, termination, truncation, info = env.last()

#     if termination or truncation:
#         action = None
#     else:
#         # this is where you would insert your policy
#         action = env.action_space(agent).sample()

#     env.step(action)
# env.close()

In [ ]:
# parser = add_rllib_example_script_args(
# default_iters=2,
# default_timesteps=10000,
# default_reward=0.0,
# )


In [ ]:

# parser.add_argument(
# "--use-onnx-for-inference",
# action="store_true",
# help="Whether to convert the loaded module to ONNX format and then perform "
# "inference through this ONNX model.",
# )
# parser.add_argument(
# "--explore-during-inference",
# action="store_true",
# help="Whether the trained policy should use exploration during action "
# "inference.",
# )
# parser.add_argument(
# "--num-episodes-during-inference",
# type=int,
# default=10,
# help="Number of episodes to do inference over (after restoring from a checkpoint).",
# )


_StoreAction(option_strings=['--num-episodes-during-inference'], dest='num_episodes_during_inference', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, required=False, help='Number of episodes to do inference over (after restoring from a checkpoint).', metavar=None)

In [ ]:

# args = parser.parse_args()


In [ ]:

# print(args)

Namespace(algo='PPO', enable_new_api_stack=True, framework='torch', env=None, num_env_runners=None, num_envs_per_env_runner=None, num_agents=4, evaluation_num_env_runners=0, evaluation_interval=0, evaluation_duration=10, evaluation_duration_unit='episodes', evaluation_parallel_to_training=False, output=None, log_level=None, no_tune=False, num_samples=1, max_concurrent_trials=None, verbose=2, checkpoint_freq=1, checkpoint_at_end=True, wandb_key=None, wandb_project=None, wandb_run_name=None, stop_reward=200.0, stop_iters=2, stop_timesteps=10000, as_test=False, as_release_test=False, num_learners=None, num_cpus_per_learner=None, num_gpus_per_learner=None, num_aggregator_actors_per_learner=None, num_cpus=0, local_mode=False, num_gpus=None, use_onnx_for_inference=False, explore_during_inference=False, num_episodes_during_inference=10)


In [102]:

# assert args.num_agents > 0, "Must set --num-agents > 0 when running this script!"
# assert (
# args.enable_new_api_stack
# ), "Must set --enable-new-api-stack when running this script!"

# print(f"参数解析完成: num_agents={args.num_agents}, algo={args.algo}")

In [103]:

# # 修改环境注册，传递predator和prey的数量
# register_env("env", lambda _: PettingZooEnv(
#     waterworld_v4.env(
#         n_predators=args.n_predators,  # 需要在args中定义
#         n_prey=args.n_prey            # 需要在args中定义
#     )
# ))


In [107]:

# # 创建policies字典
# policies = {f"pursuer_{i}" for i in range(args.num_agents)}
# policies
# {p: RLModuleSpec() for p in all_policies}

In [ ]:

# base_config = (
#     get_trainable_cls(args.algo)
#     .get_default_config()
#     .environment("env")
#     .multi_agent(
#         policies=policies,
#         policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
#     )
#     .training(
#         vf_loss_coeff=0.005,
#     )
#     .rl_module(
#         rl_module_spec=MultiRLModuleSpec(
#             rl_module_specs={p: RLModuleSpec() for p in policies},
#         ),
#         model_config=DefaultModelConfig(vf_share_layers=True),
#     )
# )


In [13]:
# print(base_config.env_config)

In [ ]:

# # 训练
# print("开始训练...")
# results = run_rllib_example_script_experiment(base_config, args, keep_ray_up=True)
# print("训练完成")

2025-06-26 01:37:30,452	INFO worker.py:1747 -- Calling ray.init() again after it has already been called.


开始训练...
== Status ==
Current time: 2025-06-26 01:37:30 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_01-37-30/PPO_2025-06-26_01-37-30/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-06-26 01:37:35 (running for 00:00:05.13)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_01-37-30/PPO_2025-06-26_01-37-30/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-06-26 01:37:40 (running for 00:00:10.18)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_01-37-30/P

Trial name,env_runner_group,fault_tolerance,num_env_steps_sampled_lifetime,num_training_step_calls_per_iteration,perf,timers
PPO_env_afee0_00000,{'actor_manager_num_outstanding_async_reqs': 0},"{'num_healthy_workers': 0, 'num_remote_worker_restarts': 0}",0,1,"{'cpu_util_percent': np.float64(8.866666666666667), 'ram_util_percent': np.float64(21.366666666666664)}","{'training_iteration': 59.427974912861245, 'restore_env_runners': 0.01979821590532083, 'training_step': 59.407926392985104, 'env_runner_sampling_timer': 59.40787343050062, 'synch_env_connectors': 1.1440997599856928}"


== Status ==
Current time: 2025-06-26 01:38:36 (running for 00:01:05.60)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_01-37-30/PPO_2025-06-26_01-37-30/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




2025-06-26 01:38:37,894	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/qrbao/ray_results/PPO_2025-06-26_01-37-30' in 0.0035s.
2025-06-26 01:38:38,028	INFO tune.py:1041 -- Total run time: 67.57 seconds (67.43 seconds for the tuning loop).


== Status ==
Current time: 2025-06-26 01:38:37 (running for 00:01:07.44)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-25_23-20-00_551135_808463/artifacts/2025-06-26_01-37-30/PPO_2025-06-26_01-37-30/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+---------------------+------------+---------------------+--------+------------------+------+
| Trial name          | status     | loc                 |   iter |   total time (s) |   ts |
|---------------------+------------+---------------------+--------+------------------+------|
| PPO_env_afee0_00000 | TERMINATED | 192.168.0.25:817079 |      2 |          63.1343 |    0 |
+---------------------+------------+---------------------+--------+------------------+------+


训练完成


In [1]:
# 获取最佳结果
print("获取最佳checkpoint...")
best_result = results.get_best_result(
    metric=f"{ENV_RUNNER_RESULTS}/{EPISODE_RETURN_MEAN}", mode="max"
)

获取最佳checkpoint...


NameError: name 'results' is not defined

In [109]:
print("加载所有智能体的RLModule...")
rl_modules = {}
# 修改：使用新的智能体命名方式
predator_agents = [f"predator_{i}" for i in range(args.n_predators)]
prey_agents = [f"prey_{i}" for i in range(args.n_preys)]
all_agent_names = predator_agents + prey_agents


加载所有智能体的RLModule...


In [110]:

for agent_name in all_agent_names:
    rl_module_path = os.path.join(
        best_result.checkpoint.path,
        "learner_group",
        "learner",
        "rl_module",
        agent_name,
    )
    
    if os.path.exists(rl_module_path):
        rl_modules[agent_name] = RLModule.from_checkpoint(rl_module_path)
        print(f"成功加载 {agent_name} 的模型")
    else:
        print(f"警告: 找不到 {agent_name} 的模型路径: {rl_module_path}")
print(f"总共加载了 {len(rl_modules)} 个智能体模型")

成功加载 predator_0 的模型
成功加载 predator_1 的模型
成功加载 prey_0 的模型
成功加载 prey_1 的模型
总共加载了 4 个智能体模型


In [111]:

# 推理阶段
print("开始推理...")

def get_action_from_rl_module(rl_module, observation, explore=False):
    """从RLModule获取动作"""
    # 将观察转换为torch tensor并添加batch维度
    input_dict = {Columns.OBS: torch.from_numpy(observation).unsqueeze(0)}
    
    if explore:
        # 使用探索性前向传播
        rl_module_out = rl_module.forward_exploration(input_dict)
        action_dist_inputs = rl_module_out["action_dist_inputs"][0]
        action = TorchDiagGaussian.from_logits(action_dist_inputs.unsqueeze(0)).sample().squeeze(0).numpy()
    else:
        # 使用推理前向传播
        rl_module_out = rl_module.forward_inference(input_dict)
        action_dist_inputs = rl_module_out["action_dist_inputs"][0]
        action = TorchDiagGaussian.from_logits(action_dist_inputs.unsqueeze(0)).sample().squeeze(0).numpy()
    
    return action

开始推理...


In [112]:

# 运行推理episodes
for episode in range(args.num_episodes_during_inference):
    print(f"\n=== Episode {episode + 1} ===")
    
    # 创建环境（使用新的参数方式）
    env = waterworld_v4.env(
        render_mode="human",
        n_predators=args.n_predators,  # 修改：使用新的参数
        n_preys=args.n_preys,         # 修改：使用新的参数
        n_evaders=5,  # 可以根据需要调整
        n_poisons=10,  # 可以根据需要调整
    )
    env.reset(seed=42 + episode)
    
    episode_rewards = {agent: 0 for agent in env.agents}
    step_count = 0
    
    try:
        for agent in env.agent_iter():
            observation, reward, termination, truncation, info = env.last()
            
            # 累积奖励
            if agent in episode_rewards:
                episode_rewards[agent] += reward
            
            if termination or truncation:
                action = None
                print(f"{agent} 终止, 奖励: {reward}")
            else:
                # 使用对应的RLModule获取动作
                if agent in rl_modules:
                    try:
                        action = get_action_from_rl_module(
                            rl_modules[agent], 
                            observation, 
                            explore=args.explore_during_inference
                        )
                        print(f"{agent} 执行动作: {action}")
                    except Exception as e:
                        print(f"为 {agent} 获取动作时出错: {e}")
                        # 如果出错，使用随机动作作为备选
                        action = env.action_space(agent).sample()
                        print(f"{agent} 使用随机动作: {action}")
                else:
                    # 如果没有找到对应的模型，使用随机动作
                    action = env.action_space(agent).sample()
                    print(f"{agent} 没有找到对应模型，使用随机动作: {action}")
            
            env.step(action)
            step_count += 1
            
            # 每100步输出一次进度
            if step_count % 100 == 0:
                print(f"已执行 {step_count} 步")
                
    except KeyboardInterrupt:
        print("用户中断")
        break
    except Exception as e:
        print(f"Episode {episode + 1} 出现错误: {e}")
    finally:
        env.close()
    
    # 输出episode结果
    print(f"Episode {episode + 1} 完成")
    print("各智能体总奖励:")
    for agent, total_reward in episode_rewards.items():
        print(f"  {agent}: {total_reward:.2f}")
    print(f"总步数: {step_count}")

print("推理完成!")


=== Episode 1 ===
predator_0 执行动作: [-1.8125288 -0.5591454]
[WARNING]: Received an action [-1.8125288 -0.5591454] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
predator_1 执行动作: [-0.08958572  0.6042472 ]
prey_0 执行动作: [-1.0631835  0.5505582]
[WARNING]: Received an action [-1.0631835  0.5505582] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
prey_1 执行动作: [-0.14060654 -0.10101031]
predator_0 执行动作: [-0.13771439  1.3597841 ]
[WARNING]: Received an action [-0.13771439  1.3597841 ] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
predator_1 执行动作: [0.2649867  0.99578065]
prey_0 执行动作: [1.9193748 0.5030465]
[WARNING]: Received an action [1.9193748 0.5030465] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
prey_1 执行动作: [-0.2940722  0.6914272]
predator_0 执行动作: [-0.01998153  0.6650984 ]
predator_1 执行动作: [-0.7461475  

In [ ]:
from pettingzoo.sisl import waterworld_v4

env = waterworld_v4.env(render_mode="human")
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

KeyboardInterrupt: 

: 